In [20]:
import h5py    
import numpy as np  
from sklearn.metrics import confusion_matrix
import torch.optim as optim
from sklearn.metrics import accuracy_score
import utils
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torch.nn as nn
import torch
from collections import OrderedDict
from openpyxl import Workbook
import openpyxl as op
import cnn_models

def loss_fn():
    criterion = nn.CrossEntropyLoss()
    return criterion

def conf_mat_create(predicted,true,correct,total_30sec_epochs,conf_mat):
    total_30sec_epochs+=true.size()[0]
    correct += predicted.eq(true.view_as(predicted)).sum().item()
    conf_mat=conf_mat+confusion_matrix(true.cpu().numpy(),predicted.cpu().numpy(),labels=classes)
    return correct, total_30sec_epochs,conf_mat

def load_model(model,path):
    state_dict_new_model=model.state_dict()
    checkpoint = torch.load(path,map_location=torch.device('cpu'))
    state_dict=checkpoint['state_dict']
    state_dict_remove_module = OrderedDict()
    for k, v in state_dict.items():
        #k = k[7:] # remove `module.`
        state_dict_remove_module[k] = v
    #print(state_dict_remove_module.keys())
    state_dict_new_model.update(state_dict_remove_module)
    model.load_state_dict(state_dict_new_model)
    return model

def test(cnn_model,data_test,epoch):
    cnn_model.eval()
    total_30sec_epochs_test = 0
    test_loss = 0.0
    correct_test = 0
    con_mat_test=np.zeros((5,5))
    lossfn=loss_fn()
    for test_idx, test_data, test_labels in data_test:
        test_data, test_labels=test_data.to(device), test_labels.to(device)
        output = cnn_model(test_data)
        test_labels_crop = test_labels.view(-1)
        test_pred = output.argmax(dim=1,keepdim=True)
        loss = lossfn(output,test_labels_crop).item()
        test_loss += test_labels_crop.size()[0]*loss
        correct_test,total_30sec_epochs_test,con_mat_test=conf_mat_create(test_pred,test_labels_crop,correct_test,total_30sec_epochs_test,con_mat_test)
    print("conf_mat_test:",con_mat_test)
    print("total_30sec_epochs_test:",total_30sec_epochs_test)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%), Epoch:{}\n'.format(test_loss/total_30sec_epochs_test, correct_test, total_30sec_epochs_test,100. * correct_test / total_30sec_epochs_test,epoch+1))
    sheet1.append([0,1,2,3,4])
    for row in con_mat_test.tolist():
        sheet1.append(row)
    
        
if __name__ == '__main__': 
    batch_size = 192
    classes=[0,1,2,3,4]
    epochs = 200
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    
    for k in range(1,2):
        path_to_cnn_model = '/RQ1/SleepEDF/WeightedLoss/models/cnn/cnn_weightedloss_outerloop10_innerloop'+str(k)+'.tar'
        path_to_hdf5_file_test = '/RQ/SleepEDF/outerloop10_test.hdf5'
        
        if os.path.isfile(path_to_results):
            wb = op.load_workbook(path_to_results)
            sheet1 = wb['Sheet 1']
        else:
            wb=Workbook()
            sheet1=wb.active
            sheet1.title = "Sheet 1"

        dataset_test = utils.my_generator1(path_to_hdf5_file_test)
        data_test = DataLoader(dataset_test, batch_size=192, shuffle=False, sampler=None,batch_sampler=None)

        cnn_model = cnn_models.SleepMultiChannelNet(lstm_option=False) 
        model = load_model(cnn_model, path_to_cnn_model)
        test(cnn_model,data_test,epoch=0)
        wb.save(path_to_results)

Total Length instances in generator: 2809
conf_mat_test: [[ 497.   45.   11.    7.    4.]
 [  63.   89.   10.    0.   28.]
 [  10.   34. 1061.   70.   92.]
 [   0.    0.    5.  165.    0.]
 [  33.   79.    6.    2.  498.]]
total_30sec_epochs_test: 2809

Test set: Average loss: 0.5017, Accuracy: 2310/2809 (82%), Epoch:1

Total Length instances in generator: 2809
conf_mat_test: [[4.530e+02 6.800e+01 2.100e+01 1.500e+01 7.000e+00]
 [3.900e+01 8.500e+01 2.800e+01 1.000e+00 3.700e+01]
 [6.000e+00 1.600e+01 1.079e+03 9.600e+01 7.000e+01]
 [0.000e+00 0.000e+00 1.000e+00 1.690e+02 0.000e+00]
 [2.900e+01 5.600e+01 2.700e+01 5.000e+00 5.010e+02]]
total_30sec_epochs_test: 2809

Test set: Average loss: 0.5066, Accuracy: 2287/2809 (81%), Epoch:1

Total Length instances in generator: 2809
conf_mat_test: [[ 477.   45.   15.   23.    4.]
 [  48.   67.   20.    0.   55.]
 [   6.   17. 1054.   73.  117.]
 [   0.    0.    4.  166.    0.]
 [  22.   58.   10.    3.  525.]]
total_30sec_epochs_test: 2809

Te